### Postprocessing Results
Analyses were run off in 50 'buckets' of 1,000 (load cases).  This was done to enable the analsyes to be run in parallel across several machines. Timetraces for 6 parameters were generated for each analysis. These timetraces need to post-processed to get the max range for each parameter.  A summary dataframe for each bucket (1,000 files) will be stored in the analysis results sub-folder.  

In [89]:
import glob as gb
import ntpath # Equivalent of R's base (remove path)
import pandas as pd

In [90]:
# Results will be processed a the become available so let's process one at a time
# Specify the folder where the results are located
BUCKET = 'bucket23'
#local_dir = f'/Users/jasonpayne/Documents/slu_res/{BUCKET}/'
local_dir = f'C:\\Flexcom_Analysis\\_slug_project\\{BUCKET}\\'
plot_files = gb.glob(f'{local_dir}*.mplt')

In [91]:
# Create a list of load case references that are covered by the timetrace files
lc_files = []
for file_p in plot_files:
    # Remove the last 8 chars from the plot file name (this will be the load case ref.)
    lc_files.append(ntpath.basename(file_p[:len(file_p) - 8]))

# Remove duplicates & sort
lc_files = list(dict.fromkeys(lc_files))
lc_files.sort()

In [92]:
# Create empty lists of take processed results
load_cases = []
out_verDisp_mm_n5 = []
out_oopDisp_mm_n5 = []
out_oopDisp_mm_n2 = []
out_endBend_kNm_n6 = []
out_supReac_kN_n3 = []
out_supReac_kN_n4 = []

# Loop through and process each load case & each plot file in each load case
for lc in lc_files:
    load_cases.append(lc)
    for tt in ['T1', 'T3', 'T4', 'T5', 'T6', 'T7']:
        # Reconstruct the filename from the load case reference
        file_p = f'{lc}.{tt}.mplt'
        
        # Read the time trace file (skip header & note unusual delimeter)
        plot_raw = pd.read_csv(f'{local_dir}{file_p}', skiprows=12, delimiter='"')
        dat_dt = {'time':plot_raw.iloc[:,3].dropna(), 'value':plot_raw.iloc[:,5].dropna()}
        dat_df = pd.DataFrame(dat_dt)

        # Breakdown of types of timetrace
        motions = ['T1', 'T2', 'T3', 'T4']
        moments = ['T5', 'T6']
        reactin = ['T7']

        # Apply some unit conversions depending on type
        # Use 'mm' for displacement ranges
        if (any(x in file_p for x in motions)):
            dat_df.value = dat_df.value * 1000

        elif (any(x in file_p for x in moments)):
            dat_df.value = dat_df.value / 1000

        elif (any(x in file_p for x in reactin)):
            dat_df.value = dat_df.value / 1000

        # Calculate the range
        dat_range = dat_df.value.max() - dat_df.value.min()
        #print(file_p[:len(file_p) - 8])
        #print(f'Max: {round(dat_df.value.max(), 1)}, Min: {round(dat_df.value.min(), 1)}') 
        #print('Range: ', round(dat_range, 1), '\n')

        # Collect the results
        if 'T1' in file_p:
            out_verDisp_mm_n5.append(round(dat_range, 2))
        elif 'T3' in file_p:
            out_oopDisp_mm_n5.append(round(dat_range, 2))
        elif 'T4' in file_p:
            out_oopDisp_mm_n2.append(round(dat_range, 2))
        elif 'T5' in file_p:
            out_endBend_kNm_n6.append(round(dat_range, 2))
        elif 'T6' in file_p:
            out_supReac_kN_n3.append(round(dat_range, 2)) 
        elif 'T7' in file_p:
            out_supReac_kN_n4.append(round(dat_range, 2))     
    

In [93]:
# Now compile the results in a single dataframe
# ver: Vertical
# oop: Out of plane
# disp: Displacement
# Reac: Reaction
res_df = pd.DataFrame(load_cases, columns = ['load_case'])
res_df['out_oopDisp_mm_n2'] = out_oopDisp_mm_n2
res_df['out_supReac_kN_n3'] = out_supReac_kN_n3
res_df['out_supReac_kN_n4'] = out_supReac_kN_n4
res_df['out_verDisp_mm_n5'] = out_verDisp_mm_n5
res_df['out_oopDisp_mm_n5'] = out_oopDisp_mm_n5
res_df['out_endBend_kNm_n6'] = out_endBend_kNm_n6
res_df

,load_case,out_oopDisp_mm_n2,out_supReac_kN_n3,out_supReac_kN_n4,out_verDisp_mm_n5,out_oopDisp_mm_n5,out_endBend_kNm_n6
0,22001_sd843_sl14_pd177_pl10_sv9_bp12_ba770_bp52,196.45,19.84,61.66,296.68,206.30,292.17
1,22002_sd659_sl9_pd218_pl12_sv6_bp13_ba760_bp65,203.66,15.94,53.21,249.86,221.56,213.61
2,22003_sd725_sl9_pd295_pl11_sv6_bp11_ba770_bp50,179.66,14.72,53.29,220.23,176.47,167.43
3,22004_sd755_sl8_pd161_pl9_sv10_bp13_ba640_bp53,192.67,16.13,50.36,197.63,211.24,224.23
4,22005_sd588_sl10_pd248_pl8_sv8_bp13_ba620_bp69,149.03,13.72,45.10,174.00,137.06,151.04
...,...,...,...,...,...,...,...
995,22996_sd830_sl9_pd133_pl10_sv8_bp11_ba740_bp55,199.83,16.06,53.02,237.75,207.13,224.90
996,22997_sd703_sl11_pd273_pl12_sv7_bp12_ba730_bp53,172.38,14.85,50.71,215.20,168.60,182.48
997,22998_sd630_sl12_pd179_pl12_sv3_bp13_ba660_bp50,155.03,13.68,47.55,200.96,135.00,143.33
998,22999_sd702_sl11_pd239_pl8_sv11_bp12_ba620_bp62,155.07,14.86,55.04,229.43,159.51,257.13


In [94]:
# Write the results as a csv
res_df.to_csv(f'{BUCKET}_results.csv', index=False)